# Matrix of linear forms

# Approximating functions via repeated integration

Copyright 2025 Stephan Kulla ("Kulla")

Licensed under the Apache License, Version 2.0 (the "Apache License") and Creative Commons Attribution 4.0 International (the "CC-BY License"). You may choose either of these licenses to govern your use of this project.

You may obtain a copy of the Apache License at:
    http://www.apache.org/licenses/LICENSE-2.0

You may obtain a copy of the CC-BY License at:
    https://creativecommons.org/licenses/by/4.0/

Unless required by applicable law or agreed to in writing, software and content distributed under the Apache License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the Apache License for the specific language governing permissions and limitations under the License.

Under the CC-BY License, you are free to share and adapt the material provided you give appropriate credit, provide a link to the license, and indicate if changes were made. See the CC-BY License for full details.

Third-Party Components and Licenses:
This product also includes third-party components which are distributed under their respective licenses. Please refer to the NOTICE file for details.

## Utility functions and imports

In [3]:
%load_ext jupyter_ai_magics

In [4]:
import string
import sympy as sp

from IPython.display import display, Markdown

class LatexFormatter(string.Formatter):
    def format_field(self, value, format_spec):
        if format_spec == 'tex':
            return sp.printing.latex(value)
        if format_spec == '$' or format_spec == 'inline':
            return "$" + sp.printing.latex(value) + "$"
        elif format_spec == '$$' or format_spec == 'block':
            return "$$" + sp.printing.latex(value) + "$$"
        else:
            return super().format_field(value, format_spec)

def disp_md(template, **kwargs):
    display(Markdown(LatexFormatter().format(template, **kwargs)))

## Definition of inner product and integral form on $C([a,b])$

In [27]:
from sympy.abc import x

def inner_product(f, g, a=0, b=1):
    return sp.integrate(f * g, (x, a, b))

inner_product(sp.Function("f")(x), sp.Function("g")(x))

Integral(f(x)*g(x), (x, 0, 1))

In [14]:
# Tests

assert inner_product(sp.S(1), sp.S(1)) == sp.S(1)
assert inner_product(x, x**2) == sp.S(1) / 4

In [17]:
def linear_form(f, a=0, b=1):
    def returned_function(g, a=a, b=b):
        return inner_product(f, g, a=a, b=b)
    return returned_function

assert linear_form(x)(x**2) == sp.S(1)/4

## Calculate an approximation of a function based on linear measures

Lets assume we have $n$ linear independent measures $M_n$ for (continuious) functions $f: [a,b]\to \mathbb R$. Given we have $n$ linear independent base functions $b_k:[a,b]\to\mathbb R$.

Then we can calculate an approximation $a=\sum_{k=1}^n c_k b_k$ such that $\forall k\in\{1,2,\ldots n\}: M_k(a) = M_k(f)$. This can be reduced to the linear system of equations: $$\forall j \in \{1,2,\ldots n\}: \sum_{i=1}^n c_i M_j(b_i) = M_j(f)$$

This can be represented as $M \vec c = \vec f$ with $\vec c = (c_1, \ldots, c_n)$, $\vec f = (M_1(f), \ldots, M_n(f))$ and $M$ is a matrix with $M_ij = M_i(b_j)$. We call $M$ the `matrix_of_linearform`.

In [26]:
from typing import List, Callable

from sympy.abc import n

def linear_forms(expr: sp.Expr, n: int):
    return [linear_form(expr.subs({"n": i})) for i in range(n)]

def vectors(expr: sp.Expr, n: int):
    return [expr.subs({"n": i}) for i in range(n)]

def matrix_of_linear_forms(
    linear_forms: List[Callable[[sp.Expr], sp.Number]],
    vectors: List[sp.Expr]
) -> sp.Matrix:
    return sp.Matrix([[M(b) for b in vectors] for M in linear_forms])

matrix_of_linear_forms(linear_forms(x**n, 3), vectors(x**n, 3))

Matrix([
[  1, 1/2, 1/3],
[1/2, 1/3, 1/4],
[1/3, 1/4, 1/5]])

In [30]:
def derivative(n):
    def returned_function(f):
        if n == 0:
            return f.subs(x, 0)
        else:
            return sp.diff(f, x, n).subs(x, 0)
    return returned_function

def derivatives(n):
    return [derivative(i) for i in range(n)]

matrix_of_linear_forms(derivatives(5), vectors(x**n, 5))

Matrix([
[1, 0, 0, 0,  0],
[0, 1, 0, 0,  0],
[0, 0, 2, 0,  0],
[0, 0, 0, 6,  0],
[0, 0, 0, 0, 24]])